In [120]:
import pickle
import os
from pathlib import Path
from algo_2 import *

In [121]:
# List all files in the directory that start with "output "
directory = os.getcwd()
pickle_files = [f for f in os.listdir(directory) if f.startswith("output ")]

# Initialize a dictionary to store the contents of each pickle file
pickle_contents = []

# Loop through each file, read its content, and store it in the dictionary
for file_name in pickle_files:
    file_path = os.path.join(directory, file_name)
    with open(file_path, 'rb') as file:
        pickle_contents.extend(pickle.load(file))

# pickle_contents.keys()
pickle_contents.sort(key=lambda x: x[0][1], reverse=True)
all_tickers = [i[0][0]["ticker"] for i in pickle_contents]
len(all_tickers)

292

In [122]:
finer_pick = []
a = []
b=[]
c=[]
finer_finer_pick = []
for i in pickle_contents:
    for j in range(len(i)):
        stock = algo2(**i[j][0])
        # eval(stock, opt=False)
        maximum_drawdown, ROI, win_loss_ratio = eval(stock, opt=True)
        buy_orders = []
        sell_orders = []
        current_order = None
        money = 100000
        short = None
        long = None

        for index, row in stock.iterrows():
            if row['order'] > 0:
                if current_order is None and short is None:
                    current_order = {"Initial Money": money, 'Buy Date': row['date'], 'Buy Price': stock.loc[index -
                                                                                                            1, 'close'], "Shares Purchased": money/stock.loc[index - 1, 'close'], "long/short": "long"}
                    long = True
                elif current_order is not None and short is not None:
                    current_order['Sell Date'] = row['date']
                    current_order['Sell Price'] = row['close']
                    current_order["Price Change"] = current_order["Sell Price"] - \
                        current_order["Buy Price"]
                    current_order["Profit"] = current_order["Price Change"] * \
                        current_order["Shares Purchased"] * -1
                    current_order["Money After Order"] = current_order["Shares Purchased"] * \
                        current_order['Buy Price'] + current_order["Profit"]
                    money = current_order["Money After Order"]
                    sell_orders.append(current_order)
                    short = None
                    current_order = {"Initial Money": money, 'Buy Date': row['date'], 'Buy Price': row[
                        'close'], "Shares Purchased": money/row['close'], "long/short": "long"}
                    long = True
                else:
                    continue
            elif row['order'] < 0:
                if current_order is None and long is None:
                    current_order = {"Initial Money": money, 'Buy Date': row['date'], 'Buy Price': stock.loc[index -
                                                                                                            1, 'close'], "Shares Purchased": money/stock.loc[index - 1, 'close'], "long/short": "short"}
                    short = True
                elif current_order is not None and long is not None:
                    current_order['Sell Date'] = row['date']
                    current_order['Sell Price'] = row['close']
                    current_order["Price Change"] = current_order["Sell Price"] - \
                        current_order["Buy Price"]
                    current_order["Profit"] = current_order["Price Change"] * \
                        current_order["Shares Purchased"]
                    current_order["Money After Order"] = current_order["Shares Purchased"] * \
                        current_order['Buy Price'] + current_order["Profit"]
                    money = current_order["Money After Order"]
                    sell_orders.append(current_order)
                    long = None
                    current_order = {"Initial Money": money, 'Buy Date': row['date'], 'Buy Price': row[
                        'close'], "Shares Purchased": money/row['close'], "long/short": "short"}
                    short = True
                else:
                    continue
            elif row['order'] == 0:
                if current_order is not None:
                    current_order['Sell Date'] = row['date']
                    current_order['Sell Price'] = row['close']
                    current_order["Price Change"] = current_order["Sell Price"] - \
                        current_order["Buy Price"]
                    if current_order['long/short'] == "short":
                        current_order["Profit"] = current_order["Price Change"] * \
                            current_order["Shares Purchased"] * -1
                    elif current_order['long/short'] == "long":
                        current_order["Profit"] = current_order["Price Change"] * \
                            current_order["Shares Purchased"]
                    current_order["Money After Order"] = current_order["Shares Purchased"] * \
                        current_order['Buy Price'] + current_order["Profit"]
                    money = current_order["Money After Order"]
                    sell_orders.append(current_order)
                    current_order = None
                    long = None
                    short = None

        # Create a new DataFrame for buy and sell orders
        orders_df = pd.DataFrame(buy_orders + sell_orders)
        if len(orders_df) >= 18:
            if i[j][0]["ticker"] not in a:
                a.append(i[j][0]["ticker"])
            if ROI >= 2 and maximum_drawdown <= 0.12 and win_loss_ratio >= 0.75:
                if i[j][0]["ticker"] not in finer_pick:
                    finer_pick.append(i[j][0]["ticker"])
            if ROI >= 2 and maximum_drawdown <= 0.10 and win_loss_ratio >= 0.78:
                if i[j][0]["ticker"] not in b:
                    b.append(i[j][0]["ticker"])
            if ROI >= 2 and maximum_drawdown <= 0.15 and win_loss_ratio >= 0.8:
                if i[j][0]["ticker"] not in c:
                    c.append(i[j][0]["ticker"])
            if ROI >=5 or maximum_drawdown <= 0.06 or win_loss_ratio >= 0.85:
                if i[j][0]["ticker"] not in finer_finer_pick:
                    finer_finer_pick.append(i[j][0]["ticker"])

In [123]:
len(set(a))

263

In [124]:
data_path = Path(os.getcwd())
path = data_path / 'stock price data'
finer_pick = set(finer_pick)
ticker_all = pd.read_csv(path/"finer_data_complete_stock.csv")
ticker_finer = ticker_all[ticker_all["Symbol"].isin(finer_pick)]
ticker_finer.groupby("Sector")["Sector"].count()

Sector
Basic Materials            9
Communication Services     5
Consumer Cyclical         17
Consumer Defensive         6
Energy                     3
Financial Services        11
Healthcare                 9
Industrials               17
Real Estate                2
Technology                17
Utilities                  1
Name: Sector, dtype: int64

In [125]:
ticker_a = ticker_all[ticker_all["Symbol"].isin(a)]
ticker_a.groupby("Sector")["Sector"].count()

Sector
Basic Materials           16
Communication Services    11
Consumer Cyclical         39
Consumer Defensive         9
Energy                    18
Financial Services        34
Healthcare                26
Industrials               47
Real Estate                7
Technology                53
Utilities                  3
Name: Sector, dtype: int64

In [126]:
ticker_b = ticker_all[ticker_all["Symbol"].isin(b)]
ticker_b.groupby("Sector")["Sector"].count()

Sector
Basic Materials            2
Communication Services     1
Consumer Cyclical         12
Consumer Defensive         3
Energy                     1
Financial Services         6
Healthcare                 3
Industrials                8
Technology                13
Utilities                  1
Name: Sector, dtype: int64

In [127]:
ticker_c = ticker_all[ticker_all["Symbol"].isin(c)]
ticker_c.groupby("Sector")["Sector"].count()

Sector
Basic Materials            5
Communication Services     3
Consumer Cyclical         14
Consumer Defensive         4
Energy                     4
Financial Services        11
Healthcare                 7
Industrials               12
Real Estate                3
Technology                16
Utilities                  1
Name: Sector, dtype: int64

In [128]:
finer_finer_pick = set(finer_finer_pick)
ticker_finer_finer = ticker_all[ticker_all["Symbol"].isin(finer_finer_pick)]
ticker_finer_finer.groupby("Sector")["Sector"].count()

Sector
Basic Materials            4
Communication Services     2
Consumer Cyclical         11
Consumer Defensive         3
Energy                     5
Financial Services        13
Healthcare                 8
Industrials               10
Real Estate                2
Technology                16
Utilities                  1
Name: Sector, dtype: int64

In [129]:
best_dict_finer = {}
best_dict_finer_finer = {}
best_dict_a = {}
for i in pickle_contents:
    if i[0][0]["ticker"] in finer_pick:
        best_dict_finer[i[0][0]["ticker"]] = i[0][0]
    if i[0][0]["ticker"] in finer_finer_pick:
        best_dict_finer_finer[i[0][0]["ticker"]] = i[0][0]
    if i[0][0]["ticker"] in a:
        best_dict_a[i[0][0]["ticker"]] = i[0][0]

best_dict_finer = pd.DataFrame(best_dict_finer).T
best_dict_finer_finer = pd.DataFrame(best_dict_finer_finer).T
best_dict_finer.describe()

,atr_len,macd_fastLen,macd_slowLen,macd_signalSmooth,macd_peakLen,gain_ratio,loss_ratio,peak2_len,peak3_len,ticker
count,97,97,97,97,97,97,97,97,97,97
unique,3,3,2,3,2,2,2,2,2,97
top,11,11,32,7,3,1,2,20,50,AMEH
freq,73,46,55,49,96,84,78,96,67,1


In [130]:
best_dict_finer_finer.describe()

,atr_len,macd_fastLen,macd_slowLen,macd_signalSmooth,macd_peakLen,gain_ratio,loss_ratio,peak2_len,peak3_len,ticker
count,75,75,75,75,75,75,75,75,75,75
unique,3,3,2,3,1,2,2,2,2,75
top,11,11,32,7,3,1,2,20,50,EVO
freq,60,33,40,34,75,63,42,74,51,1


In [131]:
best_dict_a = pd.DataFrame(best_dict_a).T
best_dict_a.describe()

,atr_len,macd_fastLen,macd_slowLen,macd_signalSmooth,macd_peakLen,gain_ratio,loss_ratio,peak2_len,peak3_len,ticker
count,263,263,263,263,263,263,263,263,263,263
unique,3,3,2,3,2,2,2,2,2,263
top,11,11,32,7,3,1,2,20,50,EVO
freq,186,119,147,126,261,202,189,261,179,1


In [132]:
with open("ticker_coarse", "wb") as fp:   #Pickling
    pickle.dump(list(set(a)), fp)
a

['EVO',
 'HWBK',
 'AMEH',
 'MBI',
 'AAL',
 'LPSN',
 'INVE',
 'DGLY',
 'DALN',
 'NG',
 'MITK',
 'TNK',
 'GLNG',
 'USPH',
 'HZO',
 'EBS',
 'UFPI',
 'ICCC',
 'CVI',
 'FOLD',
 'UNFI',
 'AEO',
 'ASGN',
 'MDGL',
 'LYTS',
 'HOV',
 'GEOS',
 'HCCI',
 'TWI',
 'GBLI',
 'AGYS',
 'CVE',
 'WNC',
 'WPRT',
 'DVAX',
 'LPX',
 'PAG',
 'TSRI',
 'MOD',
 'TNP',
 'FRPH',
 'EZPW',
 'NOTV',
 'VIAV',
 'APT',
 'EGAN',
 'RILY',
 'EPAC',
 'WHLM',
 'REX',
 'SBS',
 'LSCC',
 'PRDO',
 'TCX',
 'UFI',
 'SYNA',
 'SSRM',
 'HELE',
 'ITRI',
 'UCTT',
 'NOG',
 'MYRG',
 'IGT',
 'APEI',
 'ZEUS',
 'FONR',
 'DAL',
 'FRO',
 'FLXS',
 'EGY',
 'IVAC',
 'ECPG',
 'BH',
 'EGO',
 'PDS',
 'NSSC',
 'WWW',
 'TCOM',
 'EVC',
 'PEGA',
 'PDFS',
 'VIRC',
 'AMD',
 'LMAT',
 'VICR',
 'ULBI',
 'AVXL',
 'LMNR',
 'MGM',
 'ZION',
 'SWBI',
 'VGR',
 'MRCY',
 'ALKS',
 'GFI',
 'FBP',
 'BELFA',
 'EXTR',
 'WERN',
 'ICLR',
 'XPO',
 'AMWD',
 'MDC',
 'QDEL',
 'OLED',
 'PCYO',
 'ATGE',
 'GPRE',
 'DGII',
 'CVCO',
 'CTRA',
 'VTRS',
 'RF',
 'DHI',
 'EBIX',
 'TR',
 